In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import os
from datetime import datetime, UTC
import analytiq_data as ad
import asyncio
from bson import ObjectId
from dotenv import load_dotenv
import docrouter_app.payments

In [4]:
ad.common.setup()
await docrouter_app.payments.init_payments_env()

In [5]:
await docrouter_app.payments.delete_all_payments_customers(dryrun=False)

2025-06-30 16:16:20,642 - docrouter_app.payments - WARNING - Starting deletion of ALL Stripe customers
2025-06-30 16:16:20,647 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/customers?limit=10
2025-06-30 16:16:20,962 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers?limit=10 response_code=200
2025-06-30 16:16:20,965 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf
2025-06-30 16:16:21,177 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf response_code=200
2025-06-30 16:16:21,178 - docrouter_app.payments - INFO - Deleting subscription sub_1RfknEQ21OOWmU21uN0F2EAn for customer cus_SawcfsptWMvKKf
2025-06-30 16:16:21,179 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/subscriptions?customer=cus_SYKOqv1XPZ7lEG
2025-

{'success': True,
 'deleted_count': 36,
 'failed_count': 0,
 'errors': [],
 'warning': 'All Stripe customers have been permanently deleted.'}

In [6]:
org_id = "685c1fb8e8952690f233e049"

In [ ]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

In [ ]:
stripe_customer = await docrouter_app.payments.get_payments_customer(org_id=org_id)

In [ ]:
stripe_customer

In [ ]:
stripe_customer.id

In [ ]:
subscription = await docrouter_app.payments.get_subscription(stripe_customer.id)
if subscription:
    print(docrouter_app.payments.get_subscription_type(subscription))
else:
    print("No subscription found")

In [20]:
await docrouter_app.payments.get_stripe_usage(org_id=org_id)

2025-06-30 14:20:07,487 - docrouter_app.payments - INFO - Current period start: 2025-06-30T12:42:56
2025-06-30 14:20:07,487 - docrouter_app.payments - INFO - Current period end: 2025-07-30T12:42:56
2025-06-30 14:20:07,854 - docrouter_app.payments - INFO - Usage summary: {
  "data": [
    {
      "aggregated_value": 45.0,
      "end_time": 1753893776,
      "id": "mtrusg_test_1751301776000",
      "livemode": false,
      "meter": "mtr_test_61Sp9pf4Uc6fO2H5g41Q21OOWmU2169w",
      "object": "billing.meter_event_summary",
      "start_time": 1751301776
    }
  ],
  "has_more": false,
  "object": "list",
  "url": "/v1/billing/meters/mtr_test_61Sp9pf4Uc6fO2H5g41Q21OOWmU2169w/event_summaries"
}


{'total_usage': 45.0,
 'included_usage': 2000,
 'overage_usage': 0,
 'remaining_included': 1955.0,
 'current_period_start': datetime.datetime(2025, 6, 30, 12, 42, 56),
 'current_period_end': datetime.datetime(2025, 7, 30, 12, 42, 56),
 'period_start': datetime.datetime(2025, 6, 30, 12, 42, 56),
 'period_end': datetime.datetime(2025, 7, 30, 12, 42, 56),
 'subscription_type': 'enterprise',
 'usage_unit': 'spu'}

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [ ]:
price_id = docrouter_app.payments.TIER_TO_PRICE["individual"]
price_id

In [ ]:
# Use the new subroutine
subscription_type = docrouter_app.payments.get_subscription_type(price_id)

# Create the subscription
subscription = await docrouter_app.payments.StripeAsync.subscription_create(
    customer=stripe_customer.id,
    items=[{
        "price": price_id,
    }],
    payment_behavior="default_incomplete",
    expand=["latest_invoice.payment_intent"],
)

# Find the subscription item ID for usage reporting later
subscription_item_id = subscription["items"]["data"][0]["id"]

In [ ]:
subscription

In [17]:
await docrouter_app.payments.record_usage(org_id=org_id, spus=35, operation="test")






2025-06-30 14:19:01,422 - docrouter_app.payments - INFO - record_usage called with org_id: 685c1fb8e8952690f233e049, spus: 35, operation: test, source: backend
2025-06-30 14:19:01,764 - docrouter_app.payments - INFO - Stripe customer found for org_id: 685c1fb8e8952690f233e049: cus_SawcfsptWMvKKf
2025-06-30 14:19:02,214 - docrouter_app.payments - INFO - Subscription item id found for org_id: 685c1fb8e8952690f233e049: si_SawgzZ0UQFvWWy
2025-06-30 14:19:02,722 - docrouter_app.payments - INFO - Reported 35 SPUs to Stripe for org 685c1fb8e8952690f233e049


2025-06-30 14:19:01,423 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27
2025-06-30 14:19:01,763 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27 response_code=200
2025-06-30 14:19:01,765 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf
2025-06-30 14:19:02,210 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf response_code=200
2025-06-30 14:19:02,215 - stripe - INFO - message='Request to Stripe api' method=post url=https://api.stripe.com/v1/billing/meter_events
2025-06-30 14:19:02,721 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/billing/meter_events response_code=200
2025-06

{'org_id': '685c1fb8e8952690f233e049',
 'stripe_customer_id': 'cus_SawcfsptWMvKKf',
 'spus': 35,
 'operation': 'test',
 'source': 'backend',
 'timestamp': datetime.datetime(2025, 6, 30, 18, 19, 2, 723727),
 'reported_to_stripe': True}

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [38]:
org_id = "6859696f6ebcee0b712b9ac6"

In [ ]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

In [ ]:
await docrouter_app.payments.delete_payments_customer(org_id=org_id, force=False)